In [2]:
%load_ext autoreload
%autoreload 2

from shapely.geometry import LineString, Point, MultiPoint, Polygon

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime
from shapely.geometry import LineString, Point, MultiPoint, Polygon
from IPython.display import display, clear_output


df = pd.read_csv('train.csv')

starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)
manh = gdf.loc[gdf.BoroName=='Manhattan'].to_crs(4326)

roads = gpd.read_file('nyc_roads/geo_export_05dcab6d-50ed-427f-aae3-61ecc2069210.shp', mask=manh)

In [101]:
from routing import route, route2
from misc import *

# i=2
for i in range(100):
    rt = route(starts.iloc[i].geometry, stops.iloc[i].geometry, roads, 50,5,2.3)
    # print(rt)
    # for k in range(5):
    #     for j in range(5):
    #         print(fastdtw([(a.x, a.y) for a in rt[k]],[(a.x, a.y) for a in rt[j]])[0])
    viz([rt[0]], starts.iloc[i:i+1], stops.iloc[i:i+1])
    input()
    clear_output()

In [94]:
viz_start_stop( starts.iloc[i:i+1], stops.iloc[i:i+1])